# TorchServe on SageMaker - BERT Japanese example

# Introduction

- [TorchServe](https://github.com/pytorch/serve)をSageMakerのModelクラスでデプロイします    
- `pytorch_training`で学習したモデルを使用します    


## IAM Role
_**Note**: IAMロールに以下の権限があることを確認してください:_

- AmazonSageMakerFullAccess
- AmazonS3FullAccess
- AmazonEC2ContainerRegistryFullAccess

ECRへイメージをpushするために、IAMに`AmazonEC2ContainerRegistryFullAccess`の権限を付与する必要があります。

## Installation

このNotebookはSageMakerの`conda_pytorch_p36`カーネルを利用して動作検証しています。

In [ ]:
!pip install --upgrade pip
!pip -q install sagemaker awscli boto3 pandas --upgrade

In [ ]:
!pip install torchserve torch-model-archiver

## Unzip a BERT model and create a TorchServe archive

- 以下コードは`pytorch_training`で学習したモデルがこのNotebookと同じ階層に配置されている前提です
    - `pytorch_training`内に学習したモデルのダウンロードコマンドがあります
- 解凍後に`pytorch_model.bin`と`config.json`が出力されていることを確認してください

In [ ]:
# Unzip

!tar -zxvf model.tar.gz

### torch model archive（.mar）を作成するためのPre-requisites：

- serialized-file(.pt)： このファイルは、eager modeモデルの場合のstate_dictを表します
- model-file(.py)： このファイルには、モデルアーキテクチャを表すtorch nn.modulesから拡張されたモデルクラスが含まれています。 このパラメーターは、eager modeモデルでは必須です。 このファイルには、torch.nn.modulesから拡張されたクラス定義が1つだけ含まれている必要があります。
- index_to_name.json： このファイルには、予測されたインデックスのクラスへのマッピングが含まれています。 デフォルトのTorchServe handlesは、予測されたインデックスと確率を返します。 このファイルは、-extra-filesパラメーターを使用してmodel archiverに渡すことができます。
- version： モデルのバージョン
- handler： TorchServeのデフォルト handlerの名前またはカスタム推論handler（.py）へのpath

**Huggingface_Transformersの場合はこちらに[sample](https://github.com/pytorch/serve/tree/master/examples/Huggingface_Transformers)があります**

- このNotebookで使用するBERT_model配下にあるファイルはsampleのコードを一部日本語用に書き換えて使用しています

In [ ]:
# TorchServe archive

!torch-model-archiver --model-name BERTJPSeqClassification --version 1.0 \
--serialized-file pytorch_model.bin \
--handler BERT_model/Transformer_handler_generalized.py \
--extra-files "config.json,BERT_model/setup_config.json,BERT_model/index_to_name.json" \
--model-file BERT_model/model.py

## Create a boto3 session and get specify a role with SageMaker access

In [ ]:
import boto3, time, json

sess = boto3.Session()
sm = sess.client('sagemaker')
region = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
import sagemaker

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

In [ ]:
bucket_name = sagemaker_session.default_bucket()
prefix = 'torchserve'
model_file_name = 'BERT_Japanese'

!tar cvfz {model_file_name}.tar.gz BERTJPSeqClassification.mar
!aws s3 cp {model_file_name}.tar.gz s3://{bucket_name}/{prefix}/models/

## Create an Amazon ECR registry

（まだ存在しなければ）torchserveコンテナイメージ用の新しいDockerコンテナレジストリを作成します。

In [ ]:
registry_name = 'torchserve-1'
!aws ecr create-repository --repository-name {registry_name}

## Build a TorchServe Docker container and push it to Amazon ECR¶


In [ ]:
%%time

image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

%cd container
!docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}
%cd ../

## Deploy endpoint and make prediction using Amazon SageMaker SDK



In [ ]:
endpoint_name = 'torchserve-endpoint-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
model_data = f's3://{bucket_name}/{prefix}/models/{model_file_name}.tar.gz'
sm_model_name = 'torchserve-bert-japanese'

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

torchserve_model = Model(
    model_data=model_data, 
    image_uri=image,
    role =role,
    predictor_cls=Predictor,
    name =sm_model_name
)

In [ ]:
predictor = torchserve_model.deploy(
    instance_type='ml.g4dn.xlarge',
    initial_instance_count=1,
    endpoint_name=endpoint_name
)

## SageMaker SDKを使用したリクエスト

In [ ]:
payload ='ハワイアンの心和む音楽の中、ちょっとシリアスなドラマが展開していきます。音楽の力ってすごいな、って思いました。'

In [ ]:
predictor.predict(data=payload).decode(encoding='utf-8')

## Boto3を使用したリクエスト

In [ ]:
'''
import boto3
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
    EndpointName='YOUR_ENDPOINT_NAME',
    Body='ハワイアンの心和む音楽の中、ちょっとシリアスなドラマが展開していきます。音楽の力ってすごいな、って思いました。',
    ContentType='text/csv',
    Accept='application/json'
)

response['Body'].read().decode(encoding='utf-8')
'''

## Endpointの削除
- 使い終わったEndpointは削除しましょう
- AmazonSageMakerのコンソールからも削除できます

In [ ]:
predictor.delete_endpoint()